<a href="https://colab.research.google.com/github/jespimentel/analisador_sentencas_2/blob/main/senten%C3%A7as_de_tr%C3%A1fico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisador de sentenças (versão Google Colab)
### José Eduardo de Souza Pimentel


In [ ]:
!pip install --upgrade litellm --q

In [ ]:
import os
import re
import json
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from litellm import completion
from google.colab import userdata

# Obtenção das sentenças

In [ ]:
# Configurações

PARAMS_TJSP = {
    "conversationId": "",
    "dadosConsulta.pesquisaLivre": "",
    "tipoNumero": "UNIFICADO",
    "numeroDigitoAnoUnificado": "",
    "foroNumeroUnificado": "",
    "dadosConsulta.nuProcesso": "",
    "dadosConsulta.nuProcessoAntigo": "",
    "classeTreeSelection.values": "8669",
    "classeTreeSelection.text": "",
    "assuntoTreeSelection.values": "3608",
    "assuntoTreeSelection.text": "",
    "agenteSelectedEntitiesList": "",
    "contadoragente": "0",
    "contadorMaioragente": "0",
    "cdAgente": "",
    "nmAgente": "",
    "dadosConsulta.dtInicio": "",
    "dadosConsulta.dtFim": "",
    "varasTreeSelection.values": "451-7101,451-10,451-9,451-7094",
    "varasTreeSelection.text": "",
    "dadosConsulta.ordenacao": "DESC"
}

PATH_SENTENCES = "/content/drive/MyDrive/Datasets/trafico_piracicaba/sentencas_baixadas"

MAX_INTERACOES = 50

PROCESSED_DATA_PATH = "/content/drive/MyDrive/Datasets/trafico_piracicaba/processadas"

CSV_SENTENCES = '/content/drive/MyDrive/Datasets/trafico_piracicaba/processadas/sentencas_baixadas.csv'

CSV_RESPOSTAS = '/content/drive/MyDrive/Datasets/trafico_piracicaba/processadas/sentencas_analisadas.csv'

In [ ]:
# Cria uma sessão persistente
session = requests.Session()

# 1a requisição para obter os cookies
response = session.get("https://esaj.tjsp.jus.br/cjpg/pesquisar.do", params=PARAMS_TJSP)

if response.status_code == 200:
    for i in range(1, MAX_INTERACOES + 1):
        pagina = str(i)
        url_pagina = f"https://esaj.tjsp.jus.br/cjpg/trocarDePagina.do?pagina={pagina}&conversationId="

        # Usa a mesma sessão para fazer a requisição
        response = session.get(url_pagina)

        if response.status_code == 200:
            # Salva o conteúdo da página
            with open(f"{PATH_SENTENCES}/pagina_{pagina}.html", "wb") as file:
                file.write(response.content)
            print(f"Página {pagina} salva com sucesso.")
        else:
            print(f"Erro ao acessar a página {pagina}: {response.status_code}")

        # Intervalo entre requisições
        time.sleep(2)
else:
    print("Erro ao acessar a página inicial.")

Página 1 salva com sucesso.
Página 2 salva com sucesso.
Página 3 salva com sucesso.
Página 4 salva com sucesso.
Página 5 salva com sucesso.
Página 6 salva com sucesso.
Página 7 salva com sucesso.
Página 8 salva com sucesso.
Página 9 salva com sucesso.
Página 10 salva com sucesso.
Página 11 salva com sucesso.
Página 12 salva com sucesso.
Página 13 salva com sucesso.
Página 14 salva com sucesso.
Página 15 salva com sucesso.
Página 16 salva com sucesso.
Página 17 salva com sucesso.
Página 18 salva com sucesso.
Página 19 salva com sucesso.
Página 20 salva com sucesso.
Página 21 salva com sucesso.
Página 22 salva com sucesso.
Página 23 salva com sucesso.
Página 24 salva com sucesso.
Página 25 salva com sucesso.
Página 26 salva com sucesso.
Página 27 salva com sucesso.
Página 28 salva com sucesso.
Página 29 salva com sucesso.
Página 30 salva com sucesso.
Página 31 salva com sucesso.
Página 32 salva com sucesso.
Página 33 salva com sucesso.
Página 34 salva com sucesso.
Página 35 salva com suc

# Criação do dataframe

In [ ]:
def extrair_informacoes_html(html_path):
    processos = []
    with open(html_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        textos = soup.find_all(class_="fundocinza1")
        for texto in textos:
            processo = {}
            # Número do Processo
            processo['numero_do_processo'] = texto.find('span', class_='fonteNegrito').text.strip()

            # Classe, Assunto, Magistrado, Comarca, Foro, Vara, Data de Disponibilização
            for row in texto.find_all('tr', class_='fonte'):
                if row.find('strong'):
                    key = row.find('strong').text.strip().replace(':', '').lower().replace(' ', '_')
                    value = row.find('strong').next_sibling.strip()
                    processo[key] = value

            # Sentença
            sentenca_divs = texto.find_all('div', align='justify')
            sentenca = ''
            for div in sentenca_divs:
                span = div.find('span')
                if span:
                    sentenca += span.text.strip() + '\n'
            processo['sentenca'] = sentenca.strip()

            processos.append(processo)

        return processos

In [ ]:
data = []

for filename in os.listdir(PATH_SENTENCES):
    if filename.endswith(".html"):
        filepath = os.path.join(PATH_SENTENCES, filename)
        data += (extrair_informacoes_html(filepath))

# Cria o DataFrame
df = pd.DataFrame(data)
display(df)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca
0,1500531-87.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,22/07/2025,
1,1500374-17.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,22/07/2025,
2,1500362-03.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,22/07/2025,
3,1506828-40.2023.8.26.0451,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,NaN,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,18/07/2025,
4,1501148-81.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,NaN,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,17/07/2025,
...,...,...,...,...,...,...,...,...,...
495,1500638-68.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,15/07/2024,
496,1500527-84.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,15/07/2024,
497,1500388-35.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Flavia de Cassia Gonzales de Oliveira,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,11/07/2024,
498,1501520-64.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Flavia de Cassia Gonzales de Oliveira,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,04/07/2024,


In [ ]:
df_1 = df[(df['sentenca']!='')]
df_1.reset_index(drop=True, inplace=True)
display(df_1)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca
0,1500585-53.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,17/07/2025,SENTENÇA\n\nProcesso Digital nº:\t1500585-53.2...
1,1500719-80.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,17/07/2025,SENTENÇA\n\nProcesso Digital nº:\t1500719-80.2...
2,1500551-78.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rodrigo Pares Andreucci,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,14/07/2025,SENTENÇA\n\nProcesso Digital nº:\t1500551-78.2...
3,1500140-35.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rodrigo Pares Andreucci,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,14/07/2025,SENTENÇA\n\nProcesso Digital nº:\t1500140-35.2...
4,1506828-40.2023.8.26.0451,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,08/07/2025,SENTENÇA\n\nProcesso Digital nº:\t1506828-40.2...
...,...,...,...,...,...,...,...,...,...
99,1500229-92.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rodrigo Pares Andreucci,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,24/07/2024,SENTENÇA\n\nProcesso Digital nº:\t1500229-92.2...
100,1500582-35.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Fabíola Giovanna Barrea Moretti,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,22/07/2024,SENTENÇA\n\nProcesso Digital nº:\t1500582-35.2...
101,1500697-56.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Fabíola Giovanna Barrea Moretti,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,22/07/2024,SENTENÇA\n\nProcesso Digital nº:\t1500697-56.2...
102,1500569-36.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Flavia de Cassia Gonzales de Oliveira,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,19/07/2024,SENTENÇA\n\nProcesso Digital nº:\t1500569-36.2...


In [ ]:
df_1.to_csv(CSV_SENTENCES, index=False, encoding='utf-8-sig')  # utf-8-sig para compatibilidade com Excel
print(f"Arquivo CSV salvo em: {CSV_SENTENCES}")

Arquivo CSV salvo em: /content/drive/MyDrive/Datasets/trafico_piracicaba/processadas/sentencas_baixadas.csv


In [ ]:
#df_1 = pd.read_csv(CSV_SENTENCES)

# Análise com LLM

In [ ]:
# Configurações

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['DEEPSEEK_API_KEY'] = userdata.get('DEEPSEEK_API_KEY')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

# modelo = 'gpt-4o-mini'
modelo = 'gpt-4.1-mini-2025-04-14'
# modelo = "deepseek/deepseek-chat"
# modelo = "gemini/gemini-2.0-flash-001"
# modelo = "gemini/gemini-2.5-flash"

path = '/content/drive/MyDrive/Datasets/sentencas_tjsp.csv'

In [ ]:
SYSTEM_MSG = (
    "Você é assessor jurídico especializado em direito penal. "
    "Responda exclusivamente em JSON válido, sem comentários, com as chaves: "
    "'quantidade_reus', 'endereco_fato', 'modus_operandi_reu',"
    "'alegacao_reu', 'resultado_processo', 'resumo_sentenca'."
)

USER_TEMPLATE = (
    "Analise a sentença a seguir e devolva um objeto JSON contendo as chaves já indicadas. "
    "Para a chave 'quantidade_reus' responda apenas com um número (ex.: 1, 2, 3 etc)"
    "Para a chave 'endereco_fato' responda com o endereço principal do fato "
    "(ex.: rua Alvorada, 123, bairro Jardim Brasília, Piracicaba/SP)."
    "Para a chave 'modus_operandi_reu', resuma em até 50 palavras de que forma o tráfico era realizado, "
    "ou escreva 'Não se aplica'"
    "Para a chave 'alegacao_reu', resuma em até 50 palavras o que o réu ou os réus alegaram em sua defesa, "
    "ou escreva 'Não se aplica'"
    "Para a chave 'resultado_processo', responda 'Procedente', 'Improcedente', 'Parcialmente procedente', "
    "ou escreva 'Não se aplica'"
    "Para a chave 'resumo_sentenca', resuma em até 100 palavras a sentença em sua totalidade.\n\n"
    "SENTENÇA:\n\n{sentenca}"
)


def analisar_sentenca_litellm(sentenca: str, modelo: str) -> dict:
  """
  Analisa o texto de uma sentença usando um modelo de linguagem
  e retorna os dados extraídos em formato de dicionário Python.
  """

  resposta_api = None
  try:
      response = completion(
          model=modelo,
          response_format={"type": "json_object"},   # força JSON limpo
          temperature=0,
          messages=[
              {"role": "system", "content": SYSTEM_MSG},
              {"role": "user", "content": USER_TEMPLATE.format(sentenca=sentenca)}
          ]
      )
      if response and response.choices and response.choices[0].message and response.choices[0].message.content:
          resposta_api = response.choices[0].message.content.strip()
      else:
          resposta_api = "Erro: Resposta da API em formato inesperado."
          print(f"Resposta inesperada da API: {response}")

  except Exception as e:
      resposta_api = f"Erro ao processar com LiteLLM ({modelo}): {e}"
      print(f"Exceção geral com o modelo {modelo}: {e}")

  return resposta_api

In [ ]:
lista_de_respostas = []
for idx, row in df_1.iterrows():
    analise = json.loads(analisar_sentenca_litellm(row['sentenca'], modelo)) # converte a string JSON para obj. Dict
    n_processo = row['numero_do_processo']
    print(f"Analisando o processo {n_processo}...")
    lista_de_respostas.append((n_processo, analise))

Analisando o processo 1500585-53.2025.8.26.0599...
Analisando o processo 1500719-80.2025.8.26.0599...
Analisando o processo 1500551-78.2025.8.26.0599...
Analisando o processo 1500140-35.2025.8.26.0599...
Analisando o processo 1506828-40.2023.8.26.0451...
Analisando o processo 1508775-32.2023.8.26.0451...
Analisando o processo 1501396-81.2023.8.26.0599...
Analisando o processo 1500186-24.2025.8.26.0599...
Analisando o processo 1500739-08.2024.8.26.0599...
Analisando o processo 1501921-29.2024.8.26.0599...
Analisando o processo 0001187-77.2025.8.26.0451...
Analisando o processo 1500212-22.2025.8.26.0599...
Analisando o processo 1501068-20.2024.8.26.0599...
Analisando o processo 1500068-48.2025.8.26.0599...
Analisando o processo 1501068-20.2024.8.26.0599...
Analisando o processo 1501094-18.2024.8.26.0599...
Analisando o processo 1500002-68.2025.8.26.0599...
Analisando o processo 1500688-94.2024.8.26.0599...
Analisando o processo 1500171-89.2024.8.26.0599...
Analisando o processo 1500123-9

In [ ]:
print(lista_de_respostas[0])

('1500585-53.2025.8.26.0599', {'quantidade_reus': 1, 'endereco_fato': 'Rua Pedro Carlos Barbosa, nº 894, bairro Monte Feliz, Piracicaba/SP', 'modus_operandi_reu': 'O réu embalava, transportava e guardava drogas em imóvel e veículo com compartimento secreto para comercialização a terceiros.', 'alegacao_reu': 'Negou inicialmente o crime, alegando invasão policial sem autorização; em juízo confessou o tráfico, alegando dificuldades financeiras e troca de arma por dívida.', 'resultado_processo': 'Procedente', 'resumo_sentenca': 'O réu Iury Nicel Gomes foi condenado por tráfico de drogas e posse ilegal de arma de fogo, com base em provas materiais, testemunhais e confissão. A abordagem policial foi considerada legal. A pena total fixada foi de cinco anos de reclusão (regime fechado), um ano de detenção (regime aberto) e quinhentos e dez dias-multa. Foram decretados o perdimento do dinheiro e do veículo apreendidos. O direito de apelar em liberdade foi vedado.'})


In [ ]:
# Cria um DataFrame a partir da lista de respostas
# A lista_de_respostas contém tuplas (numero_do_processo, dict_analise)
# Vamos expandir o dicionário em colunas do DataFrame

data_para_df = []
for numero_processo, analise_dict in lista_de_respostas:
  row_data = {'numero_do_processo': numero_processo}
  row_data.update(analise_dict) # Adiciona as chaves do dicionário como colunas
  data_para_df.append(row_data)

In [ ]:
df_respostas = pd.DataFrame(data_para_df)
display(df_respostas.head())

,numero_do_processo,quantidade_reus,endereco_fato,modus_operandi_reu,alegacao_reu,resultado_processo,resumo_sentenca
0,1500585-53.2025.8.26.0599,1,"Rua Pedro Carlos Barbosa, nº 894, bairro Monte...","O réu embalava, transportava e guardava drogas...","Negou inicialmente o crime, alegando invasão p...",Procedente,O réu Iury Nicel Gomes foi condenado por tráfi...
1,1500719-80.2025.8.26.0599,1,"Avenida Bairro Verde, altura do número 479, ba...","Réu fugiu de policiais em motocicleta, dispens...",Réu alegou desconhecer a posse da droga e arma...,Parcialmente procedente,"Réu condenado por tráfico de drogas (art. 33, ..."
2,1500551-78.2025.8.26.0599,1,"Rua Murici, n° 278, bairro Vila Sônia, Piracic...",O réu trazia consigo diversas porções de droga...,O réu alegou ser usuário de drogas e que possu...,Procedente,O réu Nelson Martins de Arruda Filho foi conde...
3,1500140-35.2025.8.26.0599,1,"Rua São Marino, nº 286, bairro Campestre, Pira...",O réu mantinha em depósito quase mil porções d...,O réu alegou que apenas guardava as drogas e o...,Procedente,O réu Kauan Washington da Silva Salvador foi c...
4,1506828-40.2023.8.26.0451,1,"avenida Gustavo Adolfo Franco Bueno, nº 674, b...",Réu guardava drogas no telhado de sua residênc...,Réu alegou que as drogas pertenciam a outro in...,Improcedente,"O réu foi denunciado por tráfico de drogas, co..."


In [ ]:
# Salvar o DataFrame em um novo arquivo CSV
df_respostas.to_csv(CSV_RESPOSTAS, index=False, encoding='utf-8-sig')
print(f"DataFrame de respostas salvo em: {CSV_RESPOSTAS}")

DataFrame de respostas salvo em: /content/drive/MyDrive/Datasets/trafico_piracicaba/processadas/sentencas_analisadas.csv
